Лабораторная работа №7. "Полносвязные нейронные сети (многослойный персептрон). Решение задач регрессии и классификации"

Задание №1.
Решить задачи регрессии и классификации на данных в соответствии с Вашим индивидуальным вариантом (см. Лаб.работы №3, 4), используя полносвязные НС; реализовать НС посредством API Keras и фреймворка TensorFlow; оценить качество полученных моделей с помощью метрик.

Задание №2.
Разработать многослойный персептрон (MLP), с помощью которого можно решать задачи регрессии и классификации. Предусмотреть возможность использования таких функции активации, как sigmoid, tanh и relu; также предусмотреть возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой. Реализовать обучение MLP методом обратного распространения ошибки; самостоятельно найти производные функций sigmoid, tanh и relu; реализовать классический градиентный спуск с возможностью указания шага.


Дополнительное Задание №3*.
1. Самостоятельно изучить отличия работы оптимизаторов Adam и RMSProp от классического градиентного спуска.
2. Реализовать градиентный спуск с использованием указанных оптимизаторов; предусмотрите возможность использования реализованных вами оптими-заторов в Вашем персептроне.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
import os

In [2]:
os.chdir('../data')
#Вариант 1
df_regression = pd.read_csv('lab3.csv')
df_classifier = pd.read_csv('lab4.csv')

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numeric_features_classifier = ['time','length'] 
df_classifier[numeric_features_classifier] = scaler.fit_transform(df_classifier[numeric_features_classifier])

In [123]:
from sklearn.model_selection import train_test_split
y_classifier = df_classifier["delay"]
X_classifier = df_classifier.drop(["delay"], axis=1)
X_classifier, _, y_classifier, _ = train_test_split(X_classifier, y_classifier, test_size=0.9, random_state=42)


y_regression = df_regression["price(euro)"]
X_regression = df_regression.drop(["price(euro)"], axis=1)

In [124]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
X_classifier, y_classifier = nm.fit_resample(X_classifier, y_classifier.ravel())

In [125]:
X_train_classifier, X_test_classifier, y_train_classifier, y_test_classifier = train_test_split(X_classifier, y_classifier, test_size=0.2)
X_train_regression, X_test_regression, y_train_regression, y_test_regression = train_test_split(X_regression, y_regression, test_size=0.2)

In [75]:
# для оценки качества решения задачи регрессии
from sklearn.metrics import mean_squared_error, mean_absolute_error
# для оценки качества решения задачи классификации
from sklearn.metrics import confusion_matrix, classification_report

In [8]:
import tensorflow as tf
import numpy as np

In [9]:
### Регрессия

In [10]:
# создаем модель, как набор последовательных слоев
model_regression = tf.keras.Sequential(
    [
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(64, activation="relu", input_shape=(939,)),
        # на втором скрытом слое будет 32 нейрона
        tf.keras.layers.Dense(32, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # на выходе один нейрон, функция активации не применяется
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [11]:
# посмотрим, какая сеть у нас получилась
model_regression.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                60160     
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 62785 (245.25 KB)
Trainable params: 62785 

In [12]:
# компилируем
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

In [13]:
X_train_regression = X_train_regression.astype('float32')
y_train_regression = y_train_regression.astype('float32')

In [14]:
# обучаем, 10 эпох означает 10 проходов по обучающей выборке
model_regression.fit(X_train_regression, y_train_regression, epochs=50, verbose=None)

In [15]:
X_test_regression = X_train_regression.astype('float32')
y_test_regression = y_train_regression.astype('float32')

In [16]:
y_pred_regression = model_regression.predict(X_test_regression)

752/752 [==============================] - 1s 952us/step


In [17]:
from sklearn.metrics import r2_score
# оцениваем качество с помощью метрик
print(mean_absolute_error(y_test_regression, y_pred_regression))
print(mean_squared_error(y_test_regression, y_pred_regression))
print(r2_score(y_test_regression, y_pred_regression))

2992.0664
15521305.0
0.43372047006974


In [18]:
### Бинарная классификация

In [19]:
model_classification = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(608,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

In [20]:
X_train_classifier = X_train_classifier.astype('float32')
y_train_classifier = y_train_classifier.astype('float32')

In [21]:
# в качестве функции активации используется бинарная  кроссэнтропия
model_classification.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
# verbose=None - не будет логов
model_classification.fit(X_train_classifier, y_train_classifier, epochs=100, verbose=None)

In [27]:
model_classification.save('models/NN_model.h5')

C:\Users\raevs\Desktop\ML\venv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
X_test_classifier = X_test_classifier.astype('float32')
y_test_classifier = y_test_classifier.astype('float32')

In [25]:
# Предсказание на тестовых данных
y_pred_classifier = [np.argmax(pred) for pred in model_classification.predict(X_test_classifier, verbose=None)]

In [26]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_classifier, y_pred_classifier)
print("Accuracy:", accuracy)

Accuracy: 0.5060422960725075


In [ ]:
print(classification_report(y_test_classifier, y_pred_classifier, zero_division=0))
print(confusion_matrix(y_test_classifier, y_pred_classifier))

Задание №2. Разработать многослойный персептрон (MLP), с помощью которого можно решать задачи регрессии и классификации. Предусмотреть возможность использования таких функции активации, как sigmoid, tanh и relu; также предусмотреть возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой. Реализовать обучение MLP методом обратного распространения ошибки; самостоятельно найти производные функций sigmoid, tanh и relu; реализовать классический градиентный спуск с возможностью указания шага.

In [117]:
import numpy as np

class MLP:
    def __init__(self, layer_sizes, activation_functions):
        self.num_layers = len(layer_sizes)
        self.layer_sizes = layer_sizes
        self.activation_functions = activation_functions
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i-1]) for i in range(1, self.num_layers)]
        self.biases = [np.random.randn(layer_sizes[i], 1) for i in range(1, self.num_layers)]

    def _sigmoid(self, x):
        return 1.0 / (1.0 + np.exp(-x))

    def _sigmoid_derivative(self, x):
        return self._sigmoid(x) * (1 - self._sigmoid(x))

    def _tanh(self, x):
        return np.tanh(x)

    def _tanh_derivative(self, x):
        return 1 - np.tanh(x)**2

    def _relu(self, x):
        return np.maximum(0, x)

    def _relu_derivative(self, x):
        return np.where(x > 0, 1, 0)

    def _feed_forward(self, x):
        activations = [x]
        zs = []
        for i in range(self.num_layers - 1):
            weight = self.weights[i]
            bias = self.biases[i]
            activation_fn = self.activation_functions[i]
            z = np.dot(weight, activations[-1]) + bias
            zs.append(z)
            if activation_fn == 'sigmoid':
                a = self._sigmoid(z)
            elif activation_fn == 'tanh':
                a = self._tanh(z)
            elif activation_fn == 'relu':
                a = self._relu(z)
            activations.append(a)
        return activations, zs

    def _backpropagate(self, x, y):
        delta_weights = [np.zeros(weight.shape) for weight in self.weights]
        delta_biases = [np.zeros(bias.shape) for bias in self.biases]
        activations, zs = self._feed_forward(x)
        delta = (activations[-1] - y)  # derivative of the loss function
        for i in range(self.num_layers - 2, -1, -1):
            z = zs[i]
            activation_fn = self.activation_functions[i]
            if activation_fn == 'sigmoid':
                derivative = self._sigmoid_derivative(z)
            elif activation_fn == 'tanh':
                derivative = self._tanh_derivative(z)
            elif activation_fn == 'relu':
                derivative = self._relu_derivative(z)
            delta_weights[i] = np.dot(delta, activations[i].T)
            delta_biases[i] = delta
            delta = np.dot(self.weights[i].T, delta) * derivative
        return delta_weights, delta_biases

    def train(self, X_train, y_train, learning_rate, num_epochs):
        for epoch in range(num_epochs):
            for x, y in zip(X_train, y_train):
                x = np.array(x, ndmin=2).T
                y = np.array(y, ndmin=2).T
                delta_weights, delta_biases = self._backpropagate(x, y)
                self.weights = [weight - learning_rate * d_weight for weight, d_weight in zip(self.weights, delta_weights)]
                self.biases = [bias - learning_rate * d_bias for bias, d_bias in zip(self.biases, delta_biases)]

    def predict(self, X_test):
        predictions = []
        for x in X_test:
            x = np.array(x, ndmin=2).T
            activations, _ = self._feed_forward(x)
            predictions.append(activations[-1])
        return predictions

In [146]:
X_train_regression = np.asarray(X_train_regression, dtype=np.float64)
y_train_regression = np.asarray(y_train_regression, dtype=np.float64).reshape(-1, 1)
# Создание объекта MLP для регрессии
mlp_regression = MLP([939, 1, 1], ['sigmoid', 'sigmoid'])

# Обучаем модель
mlp_regression.train(X_train_regression, y_train_regression, learning_rate=0.05, num_epochs=10)

X_test_regression = np.asarray(X_test_regression, dtype=np.float64)
# Получаем предсказания модели
predictions_regression = mlp_regression.predict(X_test_regression)

C:\Users\raevs\AppData\Local\Temp\ipykernel_17836\2927836980.py:12: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


In [149]:
y_test_regression = np.asarray(y_test_regression, dtype=np.float64).reshape(-1, 1)
from sklearn.metrics import r2_score
print(mean_absolute_error(y_test_regression, predictions_regression))
print(mean_squared_error(y_test_regression, predictions_regression))
print(r2_score(y_test_regression, predictions_regression))

2992.0664
15521305.0
0.43372047006974


In [154]:
X_train_classifier = np.asarray(X_train_classifier, dtype=np.float64)
y_train_classifier = np.asarray(y_train_classifier, dtype=np.float64).reshape(-1, 1)
mlp_classifier = MLP([608, 1, 1, 1], ['tanh', 'relu', 'sigmoid'])

mlp_classifier.train(X_train_classifier, y_train_classifier, learning_rate=0.05, num_epochs=10)

X_test_classifier = np.asarray(X_test_classifier, dtype=np.float64)
# Получаем предсказания модели
predictions_classifier = mlp_classifier.predict(X_test_classifier)

In [158]:
from sklearn.metrics import accuracy_score
y_test_classifier = np.asarray(y_test_classifier, dtype=np.float64)
accuracy = accuracy_score(y_test_classifier, predictions_classifier)
print("Accuracy:", accuracy)

Accuracy: 0.5060422960725075
